In [154]:
%load_ext autoreload
%autoreload 2

import pandas as pd
# pd.set_option('display.datetime_format', '%Y-%m-%d %H:%M:%S')

from code.evaluation import (compute_horizon_degradation_metrics,
                             plot_forecast_degradation,
                             plot_selected_horizons,
                             plot_forecast_trajectory,
                             compute_forecast_metrics)

from code.data_utils import clean_actual_data, clean_forecast_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
def parse_datetime(date):
    formats = ['%m/%d/%Y %I:%M:%S %p', '%m-%d-%Y %H:%M']
    # Format 1: '5/13/2025 12:00:00 AM'
    # Format 2: '05-12-2025 23:55'
    date = str(date).strip()
    for fmt in formats:
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    return pd.NaT

In [156]:
actual = pd.read_csv(r'../data/raw/actual_load.csv')
forecast = pd.read_csv(r'../data/raw/forecast_load.csv')

In [157]:
actual = actual[actual['area'] == 'PJM RTO']

In [158]:
actual = actual[['datetime_beginning_utc', 'instantaneous_load']]

In [159]:
actual = actual.rename(columns={
        'datetime_beginning_utc': 'timestamp',
        'instantaneous_load': 'load'
    })

In [160]:
actual['timestamp'] = actual['timestamp'].apply(parse_datetime)

In [161]:
# sort by timestamp
actual = actual.sort_values(by='timestamp').reset_index(drop=True)

In [162]:
forecast = forecast[forecast['forecast_area'] == 'RTO_COMBINED']

In [163]:
forecast = forecast[['evaluated_at_utc', 'forecast_datetime_beginning_utc', 'forecast_load_mw']]

In [164]:
forecast = forecast.rename(columns={
        'forecast_datetime_beginning_utc': 'target_time',
        'evaluated_at_utc': 'issued_at',
        'forecast_load_mw': 'forecast_load'
    })

In [165]:
forecast['issued_at'] = forecast['issued_at'].apply(parse_datetime)
forecast['target_time'] = forecast['target_time'].apply(parse_datetime)

In [166]:
forecast = forecast.sort_values(by=['issued_at', 'target_time']).reset_index(drop=True)

In [167]:
forecast['horizon'] = ((forecast['target_time'] - forecast['issued_at']) / pd.Timedelta(minutes=5)).astype(int)

In [168]:
latest_issued_at = forecast['issued_at'].max()
forecast = forecast[forecast['issued_at'] < latest_issued_at]

In [169]:
(forecast.shape[0])/actual.shape[0]

24.0